## Analysis of an exemplary dataset
* 3D bSSFP healthy perfusion

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

%matplotlib widget
import ipywidgets as widgets

import datetime

import os

import hypermri
import hypermri.utils.utils_anatomical as ut_anat

import sys
%load_ext autoreload
%autoreload 2

### Load data
define path and name animal

In [ ]:
# define scan path
dirpath = 'Test_Data/perfusion_test_data/'
scans = hypermri.BrukerDir(dirpath)

In [ ]:
#load PHIP data:
phip_bssfp = scans[17]
# complex seq2d:
phip_bssfp_img = phip_bssfp.Load_2dseq_file(recon_num=1)

# load DNP data:
dnp_bssfp = scans[19]
# complex seq2d:
dnp_bssfp_img = dnp_bssfp.Load_2dseq_file(recon_num=1)

# load anatomical images:
coronal = scans[18]
axial = scans[10]

print("shape of seq2d = " + str(dnp_bssfp.seq2d.shape))
print("dnp_bssfp_img = " + str(dnp_bssfp_img.shape))

#### DNP ####

In [ ]:
_, dnp_bssfp_pv_reco = dnp_bssfp.reconstruction(seq2d = dnp_bssfp.seq2d)

print("shape of dnp_bssfp_pv_reco = " + str(dnp_bssfp_pv_reco.shape))

# reorient the data so that axial and coronal match the orienatiotn of the bSSFP data:
[_, dnp_bssfp_pv_reco, axial.seq2d, _, coronal.seq2d] = dnp_bssfp.reorient_reco(
        bssfp_custom=dnp_bssfp.Reconstructed_data,    # custom reco data
        bssfp_seq2d=dnp_bssfp_pv_reco,                # paravision reconstructed data (seq2d)
        anatomical_seq2d_ax=axial.seq2d,              # axial seq2d 
        anatomical_seq2d_cor=coronal.seq2d)           # coronal seq2d

print("shape of dnp_bssfp_pv_reco = " + str(dnp_bssfp_pv_reco.shape))

In [ ]:
# shift bssfp data
dnp_bssfp_pv_reco_combined_shift = dnp_bssfp.shift_bssfp(input_data=dnp_bssfp_pv_reco, # complex, reordered bSSFP data
                                 mat_bssfp=dnp_bssfp_pv_reco.shape[2:4],                       # bssfp "axial" matrix size (phase and slice dim)
                                 mat_anat=axial.seq2d.shape[1:3],                                               # axial matrix size (phase and slice dim)
                                 fov_bssfp=dnp_bssfp.method['PVM_Fov'][1:3],                                    # bSSFP "axial" FOV (can be left out if bssfp and axial have same FOV)
                                 fov_anat=axial.method['PVM_Fov'],                                              # anatomical "axial" FOV (can be left out if bssfp and axial have same FOV)
                                 apply_fft=False,
                                                                use_scipy_shift=True)                                                                # has to be true if bSSFP data is in image space (=False if bSSFP data is in k-space)
print("shape of dnp_bssfp_pv_reco_combined_shift = " + 
      str(dnp_bssfp_pv_reco_combined_shift.shape))

#### PHIP

In [ ]:
_, phip_bssfp_pv_reco = phip_bssfp.reconstruction(seq2d = np.squeeze(phip_bssfp_img))

# reorient the data so that axial and coronal match the orienatiotn of the bSSFP data:
[_, phip_bssfp_pv_reco,anatomical_ax, anatomical_sag, anatomical_cor] = phip_bssfp.reorient_reco(
    bssfp_custom=phip_bssfp.Reconstructed_data,
    bssfp_seq2d=phip_bssfp_pv_reco,
    anatomical_seq2d_ax=axial.seq2d,
    anatomical_seq2d_cor=coronal.seq2d)

##### PHIP  - Shift the bSSFP data by subvoxels to match anatomical images:

In [ ]:
phip_bssfp_pv_reco_combined_shift = phip_bssfp.shift_bssfp(input_data=phip_bssfp_pv_reco,
                                 mat_bssfp=phip_bssfp_pv_reco.shape[2:4],
                                 mat_anat=axial.seq2d.shape[1:3],
                                 fov_bssfp=phip_bssfp.method['PVM_Fov'][1:3],
                                 fov_anat=axial.method['PVM_Fov'],
                                 apply_fft=False,use_scipy_shift=True) 

#### Plot bSSFP data.

In [ ]:
# generate figure with 3 subplots. These have to be passed into the plot3D_new2 function
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, tight_layout=True,figsize=(12,5))

# Call the plotting function
dnp_bssfp.plot3D_new2(bssfp_data=dnp_bssfp_pv_reco_combined_shift, # complex bSSFP data
                      coronal_image=coronal, # coronal mutlislice image object 
                      axial_image=axial,     # axial mutlislice image object
                      axlist=[ax1, ax2, ax3], # pass axes that the function will plot o
                      plot_params=None, # if you want to recreate plot, pass the json file name dnp_bssfp.load_plot_params(path_to_params="figures_shifted/fig_name.json")
                      fig=fig) # if you want a colorbar, this is necessary

## Interpolation of bssfp to anatomical resolution
To segment the datasets properly, they are interpolated to match the anatomical coronal resoltuion. To avoid interpolation artifacts, the interpolation method is set to "nearest" which means that no data is generated


In [ ]:
import time

# Interpolate DNP Data
start_time = time.time()

dnp_bssfp_itp = dnp_bssfp.interpolate_bssfp(bssfp_data=dnp_bssfp_pv_reco_combined_shift,
                            interp_size=(1,                              # echoes
                                         round(coronal.seq2d.shape[0]), # anatomical 
                                         round(coronal.seq2d.shape[1]), # anatomical
                                         coronal.seq2d.shape[2],        # slices 
                                         dnp_bssfp_pv_reco_combined_shift.shape[4],     # repetitions
                                         1),                             # channels
                            interp_method="nearest",                     # interpolation method
                            use_multiprocessing=True,                    # uses multiple cores
                            number_of_cpu_cores=None)                    # if =None and use_multiprocessing=True, automatically calcuates the nuber of CPU cores

print("--- %s seconds ---" % (time.time() - start_time))

# Interpolate PHIP Data
start_time = time.time()

phip_bssfp_itp = phip_bssfp.interpolate_bssfp(bssfp_data=phip_bssfp_pv_reco_combined_shift,
                            interp_size=(1,
                                         round(coronal.seq2d.shape[0]),
                                         round(coronal.seq2d.shape[1]),
                                         coronal.seq2d.shape[2],
                                         phip_bssfp_pv_reco_combined_shift.shape[4],
                                         1),
                            interp_method="nearest",
                            use_multiprocessing=True)
print("--- %s seconds ---" % (time.time() - start_time))

## We make a list of anatomical images to be segmented 
Define the ROI names that will be segmented.


In [ ]:
roi_names = ['bloodvessel', 'kidneyL','kidneyR','heart', 'muscle','phantom','outside_ref']

#### Segment the interesting anatomical ROIs

init list of segmenter objects

In [ ]:
plt.close('all')
segmenter_list = ut_anat.get_segmenter_list_overlayed(coronal.seq2d,
                                                      np.abs(dnp_bssfp_itp),
                                                      n_rois=len(roi_names),
                                                      figsize=(6,6),
                                                      overlay=0.3,vmin=0,vmax=65,                                                    
                                                      bssfp_cmap='magma')

Draw ROIs onto the coronal anatomical images. If you misdraw you can tick the Erasing box to remove your mistakes.

In [ ]:
ut_anat.draw_masks_on_anatomical(segmenter_list,roi_names)

#### Retrieve mask

In [ ]:
mask_dict = ut_anat.get_masks(segmenter_list,roi_keys=roi_names,plot_res=False)

#### Get the signal from the ROIs:

###### Split data into pyruvate and lactate

In [ ]:
dnp_bssfp_itp_pyr = dnp_bssfp_itp

phip_bssfp_itp_pyr = phip_bssfp_itp

# Mask the data first and then integrate

In [ ]:
# init empty dictonaries
dnp_signal_pyr = {}
phip_signal_pyr = {}

# calculate time curves of ROIs
for k in mask_dict:
    print(k)
    dnp_signal_pyr[k] =  dnp_bssfp.roi_signal_curve(input_data=np.abs(dnp_bssfp_itp_pyr),
                                                    mask_dict=mask_dict,
                                                    mask_key=k)

    
    phip_signal_pyr[k] =  phip_bssfp.roi_signal_curve(input_data=np.abs(phip_bssfp_itp_pyr),
                                                     mask_dict=mask_dict,
                                                     mask_key=k)

##### DNP - Plot signal time curves from ROIs

In [ ]:
# get time axis:
plt.close('all')
time_axis = dnp_bssfp.calc_timeaxis()
# Define pyruvate index (0 if you started the acquisition on the pyruvate channel)
pyr_ind = 0
plt.close()
plt.figure(figsize = (12,8),tight_layout=True)
# loop through tickers and axes
for n, k in enumerate(mask_dict.keys()):
    # filter df for ticker and plot on specified axes
    ax = plt.subplot(3, 4, n + 1)
    # plot results:
    ax.plot(time_axis,np.abs(dnp_signal_pyr[k]),label='Pyruvate')
    # chart formatting
    ax.set_xlabel("")
    ax.legend()
    ax.set_title(k)    

##### PHIP - Plot signal time curves from ROIs

In [ ]:
# get time axis:
time_axis = phip_bssfp.calc_timeaxis()
# Define pyruvate index (0 if you started the acquisition on the pyruvate channel)
pyr_ind = 0

from scipy.signal import savgol_filter

plt.figure(figsize = (12,8),tight_layout=True)
# loop through tickers and axes
for n, k in enumerate(mask_dict.keys()):

    # filter df for ticker and plot on specified axes
    ax = plt.subplot(3, 4, n + 1)
    ax.plot(time_axis,np.abs(phip_signal_pyr[k]),label='Pyruvate')   
    # chart formatting
    ax.set_xlabel("")
    ax.legend()
    ax.set_title(k)    

# ----------------------------------------------------------------
# SSI calculation
# ----------------------------------------------------------------

In [ ]:
# take a background by choosing some pixels that do not have signal
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, tight_layout=True,figsize=(12,5))

# Call the plotting function
dnp_bssfp.plot3D_new2(bssfp_data=dnp_bssfp_pv_reco_combined_shift, # complex bSSFP data
                      coronal_image=coronal, # coronal mutlislice image object 
                      axial_image=axial,     # axial mutlislice image object
                      axlist=[ax1, ax2, ax3], # pass axes that the function will plot o
                      plot_params=None, # if you want to recreate plot, pass the json file name dnp_bssfp.load_plot_params(path_to_params="figures_shifted/fig_name.json")
                      fig=fig) # if you want a colorbar, this is necessary

# Calculate a Structural Similarity Index between PHIP / DNP curves

In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from skimage import data, img_as_float
from matplotlib.patches import Rectangle
from hypermri.utils.utils_bssfp_analysis import *


# We will calculate the SSI for the sum of the range where we have signal for PHIP and DNP independently


# 1. Find the range where we have signal
* For rats this is plus minus 10 repetitions around the peak
* For mice this is minus 2 and plus 18 repetitions around the peak since there the influx of signal is much faster

In [ ]:
plt.close('all')
signal_range_dict = {'phip':[8,36],'dnp':[11,30]}
fig, (ax1, ax2, ax3) = plt.subplots(1, 3,tight_layout=True, figsize=(10,4))
dnp_bssfp.find_sig_range_reps(dnp_bssfp_pv_reco_combined_shift,
                    phip_bssfp_pv_reco_combined_shift,
                    signal_range_dict,
                    axlist = [ax1,ax2,ax3],                    
                   )


In [ ]:
ssi_dnp_sig_range_reps,ssi_phip_sig_range_reps = apply_sig_range(dnp_bssfp_pv_reco_combined_shift,
                                                                 phip_bssfp_pv_reco_combined_shift,
                                                                 dnp_bssfp.signal_range_dict)

# Use data that was only summed over the reps where we have sufficient signal 

In [ ]:
dnp_data = ssi_dnp_sig_range_reps
phip_data = ssi_phip_sig_range_reps

### Choose a background region thats suitable

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(12,5),tight_layout=True)
dnp_bssfp.norm_bssfp_to_background_and_divide_by_mean_for_SSI(dnp_data,coronal,0,axlist=ax,bg_pixels_y=[17,26])

### We can access the calculated background value as follows

In [ ]:
dnp_bg = dnp_bssfp.ssi_background_value
dnp_std = dnp_bssfp.ssi_background_std

### Repeat for PHIP

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(12,5),tight_layout=True)
phip_bssfp.norm_bssfp_to_background_and_divide_by_mean_for_SSI(phip_data,coronal,0,axlist=ax,bg_pixels_y=[17,26])


### We can access the calculated background value as follows

In [ ]:
phip_bg = phip_bssfp.ssi_background_value

### Subtract that background

In [ ]:
phip_images_min_bg = np.abs(phip_data-phip_bg)
dnp_images_min_bg = np.abs(dnp_data-dnp_bg)

### Now for each slice divide by the mean of the slice and then calculate the SSI

In [ ]:
structural_sim_indices,mean_squared_errors,snr_per_slice_dnp,snr_per_slice_phip = calculate_ssi(dnp_images_min_bg,dnp_bssfp,phip_images_min_bg,phip_bssfp)

# Chooose slices that have enough signal and take the SSI from these

In [ ]:
plt.close('all')
fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(9,3),tight_layout=True)

dnp_bssfp.find_high_snr_slices(dnp_images_min_bg,snr_per_slice_dnp,axlist = [ax1,ax2,ax3],
                               slice_default_indices_cor=[3,10],
                              slice_default_indices_ax=[6,11],
                              slice_default_indices_sag=[6,10])

# Look at same for PHIP, but we should only take slices that have enough signal in both measurements, so the values need to be the same

In [ ]:
plt.close('all')
fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(9,3),tight_layout=True)

phip_bssfp.find_high_snr_slices(phip_images_min_bg,snr_per_slice_phip,axlist = [ax1,ax2,ax3],
                               slice_default_indices_cor=[3,10],
                              slice_default_indices_ax=[6,11],
                              slice_default_indices_sag=[6,10])

# Seems okay, lets just check if we select the same slices for PHIP and DNP 

In [ ]:
if dnp_bssfp.slice_signal_axial == phip_bssfp.slice_signal_axial:
    print('Selected same axial slices, all GOOD')
else:
    print('Caution, axial slices not the same for PHIP/DNP')
if dnp_bssfp.slice_signal_coronal == phip_bssfp.slice_signal_coronal:
    print('Selected same coronal slices, all GOOD')
else:
    print('Caution, coronal slices not the same for PHIP/DNP')
if dnp_bssfp.slice_signal_sagittal == phip_bssfp.slice_signal_sagittal:
    print('Selected same sagittal slices, all GOOD')
else:
    print('Caution, sagittal slices not the same for PHIP/DNP')

# Lets take the SSI from these slices and mean it

In [ ]:
ssi_dict_lenghts = len(structural_sim_indices['coronal']),len(structural_sim_indices['axial']),len(structural_sim_indices['sagittal'])
max_len = np.max(ssi_dict_lenghts)
slices = range(max_len)

data = np.array([np.pad(structural_sim_indices['coronal'],(0,max_len-len(structural_sim_indices['coronal'])),mode='constant',constant_values = np.nan),
       np.pad(structural_sim_indices['axial'],(0,max_len-len(structural_sim_indices['axial'])),mode='constant',constant_values = np.nan),
       np.pad(structural_sim_indices['sagittal'],(0,max_len-len(structural_sim_indices['sagittal'])),mode='constant',constant_values = np.nan)]).T
ssi_all_value_df = pd.DataFrame(data, columns = ["coronal","axial","sagittal"] ,index=slices)
ssi_all_value_df

In [ ]:
ssi_slices_with_signal_reps_with_signal = ssi_all_value_df

ssi_slices_with_signal_reps_with_signal["coronal"]=ssi_slices_with_signal_reps_with_signal["coronal"][dnp_bssfp.slice_signal_coronal[0]:dnp_bssfp.slice_signal_coronal[1]+1]

ssi_slices_with_signal_reps_with_signal["axial"]=ssi_slices_with_signal_reps_with_signal["axial"][dnp_bssfp.slice_signal_axial[0]:dnp_bssfp.slice_signal_axial[1]+1]

ssi_slices_with_signal_reps_with_signal["sagittal"]=ssi_slices_with_signal_reps_with_signal["sagittal"][dnp_bssfp.slice_signal_sagittal[0]:dnp_bssfp.slice_signal_sagittal[1]+1]

ssi_slices_with_signal_reps_with_signal_output_df = pd.DataFrame(data = None ,columns = ["axial mean", "axial std",
                                                                                   "coronal mean", "coronal std",
                                                                                   "sagittal mean", "sagittal std",
                                                                                         "SSI mean", "SSI std"
                                                                                  ],index = ['SSI'])
ssi_slices_with_signal_reps_with_signal_output_df["coronal mean"] = ssi_slices_with_signal_reps_with_signal.mean(axis=0)[0]
ssi_slices_with_signal_reps_with_signal_output_df["axial mean"] = ssi_slices_with_signal_reps_with_signal.mean(axis=0)[1]
ssi_slices_with_signal_reps_with_signal_output_df["sagittal mean"] = ssi_slices_with_signal_reps_with_signal.mean(axis=0)[2]

ssi_slices_with_signal_reps_with_signal_output_df["coronal std"]= ssi_slices_with_signal_reps_with_signal.std(axis=0)[0]
ssi_slices_with_signal_reps_with_signal_output_df["axial std"]= ssi_slices_with_signal_reps_with_signal.std(axis=0)[1]
ssi_slices_with_signal_reps_with_signal_output_df["sagittal std"]= ssi_slices_with_signal_reps_with_signal.std(axis=0)[2]

ssi_slices_with_signal_reps_with_signal_output_df["SSI mean"]= np.mean([ssi_slices_with_signal_reps_with_signal_output_df["coronal mean"],
                                                                       ssi_slices_with_signal_reps_with_signal_output_df["axial mean"],
                                                                       ssi_slices_with_signal_reps_with_signal_output_df["sagittal mean"]])

ssi_slices_with_signal_reps_with_signal_output_df["SSI std"]= np.mean([ssi_slices_with_signal_reps_with_signal_output_df["coronal std"],
                                                                       ssi_slices_with_signal_reps_with_signal_output_df["axial std"],
                                                                       ssi_slices_with_signal_reps_with_signal_output_df["sagittal std"]])

ssi_slices_with_signal_reps_with_signal_output_df

# SSI mean and SSI std are the values used